In [ ]:
# Imports necesarios
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Path a los datos exportados
BASE_PATH = Path('../data_sipc/exports_dashboard')

print("✅ Librerías cargadas correctamente")
print(f"📁 Ruta de datos: {BASE_PATH.absolute()}")

## 1. Precio Promedio por Producto

Análisis de la evolución temporal de precios promedio agrupados por producto, año y mes.

In [ ]:
# Cargar datos de precio promedio
df_precio_promedio = pd.read_parquet(BASE_PATH / 'precio_promedio.parquet')

print(f"📊 Datos cargados: {len(df_precio_promedio):,} registros")
print(f"📅 Periodo: {df_precio_promedio['anio'].min()}-{df_precio_promedio['anio'].max()}")
print(f"🛒 Productos únicos: {df_precio_promedio['producto_id'].nunique()}")
print("\n🔍 Primeras filas:")
df_precio_promedio.head(10)

In [ ]:
# Seleccionar top 10 productos con más registros para visualizar
top_productos = df_precio_promedio['producto_id'].value_counts().head(10).index
df_top = df_precio_promedio[df_precio_promedio['producto_id'].isin(top_productos)].copy()

# Crear fecha completa para eje temporal
df_top['fecha'] = pd.to_datetime(df_top[['anio', 'mes']].assign(dia=1))
df_top = df_top.sort_values('fecha')

# Visualización
fig, ax = plt.subplots(figsize=(16, 8))

for producto in top_productos:
    data = df_top[df_top['producto_id'] == producto]
    ax.plot(data['fecha'], data['precio_promedio'], marker='o', linewidth=2, label=f'ID {producto}')

ax.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax.set_ylabel('Precio Promedio ($)', fontsize=12, fontweight='bold')
ax.set_title('Evolución de Precios Promedio - Top 10 Productos', fontsize=14, fontweight='bold', pad=20)
ax.legend(title='Producto', bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n📈 Estadísticas generales:")
print(df_precio_promedio.groupby('anio')['precio_promedio'].agg(['mean', 'median', 'min', 'max']).round(2))

## 2. Variación Porcentual Mensual

Cambios porcentuales en precios comparando mes actual vs. mes anterior.

In [ ]:
# Cargar datos de variación mensual
df_variacion = pd.read_parquet(BASE_PATH / 'variacion_mensual.parquet')

print(f"📊 Datos cargados: {len(df_variacion):,} registros")
print(f"\n🔍 Estadísticas de variación:")
print(df_variacion['variacion_porcentual'].describe())
print("\n📉 Productos con mayor caída de precio:")
print(df_variacion.nsmallest(5, 'variacion_porcentual')[['producto_id', 'anio', 'mes', 'variacion_porcentual']])
print("\n📈 Productos con mayor aumento de precio:")
print(df_variacion.nlargest(5, 'variacion_porcentual')[['producto_id', 'anio', 'mes', 'variacion_porcentual']])

In [ ]:
# Filtrar variaciones extremas (outliers) para mejor visualización
df_var_filtrado = df_variacion[
    (df_variacion['variacion_porcentual'] >= -50) & 
    (df_variacion['variacion_porcentual'] <= 50)
].copy()

# Histograma de distribución de variaciones
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histograma
ax1.hist(df_var_filtrado['variacion_porcentual'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Sin variación')
ax1.set_xlabel('Variación Porcentual (%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax1.set_title('Distribución de Variaciones de Precio Mensuales', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Boxplot por año
df_var_filtrado['fecha'] = pd.to_datetime(df_var_filtrado[['anio', 'mes']].assign(dia=1))
sns.boxplot(data=df_var_filtrado, x='anio', y='variacion_porcentual', ax=ax2, palette='Set2')
ax2.axhline(0, color='red', linestyle='--', linewidth=2, alpha=0.5)
ax2.set_xlabel('Año', fontsize=12, fontweight='bold')
ax2.set_ylabel('Variación Porcentual (%)', fontsize=12, fontweight='bold')
ax2.set_title('Variación de Precios por Año', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Precio Mínimo y Máximo

Rango de precios (mínimo y máximo) observados para cada producto en el periodo analizado.

In [ ]:
# Cargar datos de min/max precios
df_min_max = pd.read_parquet(BASE_PATH / 'min_max_precios.parquet')

print(f"📊 Datos cargados: {len(df_min_max):,} registros")
print(f"\n🔍 Primeras filas:")
df_min_max.head(10)

In [ ]:
# Calcular rango de precio (diferencia entre max y min)
df_min_max['rango_precio'] = df_min_max['precio_maximo'] - df_min_max['precio_minimo']
df_min_max['fecha'] = pd.to_datetime(df_min_max[['anio', 'mes']].assign(dia=1))

# Seleccionar top 15 productos con mayor rango de precio
top_rangos = df_min_max.nlargest(15, 'rango_precio')

# Visualización
fig, ax = plt.subplots(figsize=(16, 8))

x = range(len(top_rangos))
width = 0.35

ax.bar([i - width/2 for i in x], top_rangos['precio_minimo'], width, label='Precio Mínimo', color='lightcoral', alpha=0.8)
ax.bar([i + width/2 for i in x], top_rangos['precio_maximo'], width, label='Precio Máximo', color='lightblue', alpha=0.8)

ax.set_xlabel('Producto ID - Fecha', fontsize=12, fontweight='bold')
ax.set_ylabel('Precio ($)', fontsize=12, fontweight='bold')
ax.set_title('Top 15 Productos con Mayor Rango de Precio (Min vs Max)', fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels([f"{row['producto_id']}\n{row['anio']}-{row['mes']:02d}" for _, row in top_rangos.iterrows()], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📊 Productos con mayor variabilidad de precio:")
print(top_rangos[['producto_id', 'anio', 'mes', 'precio_minimo', 'precio_maximo', 'rango_precio']].to_string(index=False))

## 4. Costo de Canasta Básica por Supermercado

Análisis del costo de la Canasta Básica Alimentaria (CBAEN 2024) en diferentes cadenas de supermercados.

In [ ]:
# Cargar datos de canasta básica
df_canasta = pd.read_parquet(BASE_PATH / 'canasta_basica.parquet')

print(f"📊 Datos cargados: {len(df_canasta):,} registros")
print(f"🏪 Cadenas analizadas: {df_canasta['cadena_normalizada'].nunique()}")
print(f"📅 Periodo: {df_canasta['anio'].min()}-{df_canasta['anio'].max()}")
print("\n🔍 Primeras filas:")
df_canasta.head(10)

In [ ]:
# Crear fecha y ordenar
df_canasta['fecha'] = pd.to_datetime(df_canasta[['anio', 'mes']].assign(dia=1))
df_canasta = df_canasta.sort_values('fecha')

# Obtener las cadenas con más datos
top_cadenas = df_canasta['cadena_normalizada'].value_counts().head(8).index
df_canasta_top = df_canasta[df_canasta['cadena_normalizada'].isin(top_cadenas)]

# Visualización: Evolución temporal del costo de canasta
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))

# Gráfico 1: Series temporales
for cadena in top_cadenas:
    data = df_canasta_top[df_canasta_top['cadena_normalizada'] == cadena]
    ax1.plot(data['fecha'], data['costo_canasta'], marker='o', linewidth=2, label=cadena, alpha=0.8)

ax1.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax1.set_ylabel('Costo Canasta Básica ($)', fontsize=12, fontweight='bold')
ax1.set_title('Evolución del Costo de Canasta Básica por Cadena', fontsize=14, fontweight='bold', pad=20)
ax1.legend(title='Cadena', bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# Gráfico 2: Boxplot comparativo
sns.boxplot(data=df_canasta_top, x='cadena_normalizada', y='costo_canasta', ax=ax2, palette='Set3')
ax2.set_xlabel('Cadena de Supermercados', fontsize=12, fontweight='bold')
ax2.set_ylabel('Costo Canasta Básica ($)', fontsize=12, fontweight='bold')
ax2.set_title('Distribución de Costos por Cadena', fontsize=14, fontweight='bold', pad=20)
ax2.grid(True, alpha=0.3, axis='y')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Estadísticas por cadena
print("\n📊 Estadísticas de costo de canasta por cadena:")
stats_cadena = df_canasta_top.groupby('cadena_normalizada')['costo_canasta'].agg(['mean', 'median', 'min', 'max', 'std']).round(2)
stats_cadena = stats_cadena.sort_values('mean')
print(stats_cadena)

## 5. Índice de Dispersión de Precios

Mide la variabilidad de precios en el mercado usando la fórmula: `(precio_max - precio_min) / precio_promedio`

In [ ]:
# Cargar datos de dispersión
df_dispersion = pd.read_parquet(BASE_PATH / 'dispersion_precios.parquet')

print(f"📊 Datos cargados: {len(df_dispersion):,} registros")
print(f"\n🔍 Estadísticas del índice de dispersión:")
print(df_dispersion['indice_dispersion'].describe())
print("\n📈 Productos con mayor dispersión de precios:")
print(df_dispersion.nlargest(10, 'indice_dispersion')[['producto_id', 'anio', 'mes', 'indice_dispersion']].to_string(index=False))

In [ ]:
# Filtrar outliers para mejor visualización
df_disp_filtrado = df_dispersion[df_dispersion['indice_dispersion'] <= 2.0].copy()
df_disp_filtrado['fecha'] = pd.to_datetime(df_disp_filtrado[['anio', 'mes']].assign(dia=1))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histograma de distribución
ax1.hist(df_disp_filtrado['indice_dispersion'], bins=50, color='orchid', edgecolor='black', alpha=0.7)
ax1.axvline(df_disp_filtrado['indice_dispersion'].median(), color='red', linestyle='--', linewidth=2, label=f'Mediana: {df_disp_filtrado["indice_dispersion"].median():.3f}')
ax1.set_xlabel('Índice de Dispersión', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
ax1.set_title('Distribución del Índice de Dispersión de Precios', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Evolución temporal del índice promedio
dispersion_temporal = df_disp_filtrado.groupby('fecha')['indice_dispersion'].mean().reset_index()
ax2.plot(dispersion_temporal['fecha'], dispersion_temporal['indice_dispersion'], marker='o', linewidth=2, color='darkviolet')
ax2.fill_between(dispersion_temporal['fecha'], 0, dispersion_temporal['indice_dispersion'], alpha=0.3, color='orchid')
ax2.set_xlabel('Fecha', fontsize=12, fontweight='bold')
ax2.set_ylabel('Índice de Dispersión Promedio', fontsize=12, fontweight='bold')
ax2.set_title('Evolución Temporal de la Dispersión de Precios', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

## 6. Ranking de Supermercados

Comparación de cadenas de supermercados ordenadas por costo total de canasta básica.

In [ ]:
# Cargar datos de ranking
df_ranking = pd.read_parquet(BASE_PATH / 'ranking_supermercados.parquet')

print(f"📊 Datos cargados: {len(df_ranking):,} registros")
print(f"🏪 Cadenas rankeadas: {df_ranking['cadena_normalizada'].nunique()}")
print("\n🔍 Primeras filas:")
df_ranking.head(15)

In [ ]:
# Obtener el ranking más reciente
df_ranking['fecha'] = pd.to_datetime(df_ranking[['anio', 'mes']].assign(dia=1))
fecha_max = df_ranking['fecha'].max()
df_ranking_reciente = df_ranking[df_ranking['fecha'] == fecha_max].copy()

print(f"\n📅 Ranking más reciente: {fecha_max.strftime('%Y-%m')}")
print(f"🏪 Cadenas en ranking: {len(df_ranking_reciente)}")

# Ordenar por costo ascendente y tomar top 15
df_ranking_reciente = df_ranking_reciente.sort_values('costo_total').head(15)

# Visualización
fig, ax = plt.subplots(figsize=(14, 10))

colors = plt.cm.RdYlGn_r(range(len(df_ranking_reciente)))
bars = ax.barh(df_ranking_reciente['cadena_normalizada'], df_ranking_reciente['costo_total'], color=colors, edgecolor='black', alpha=0.8)

# Agregar valores al final de las barras
for i, (idx, row) in enumerate(df_ranking_reciente.iterrows()):
    ax.text(row['costo_total'] + 50, i, f"${row['costo_total']:.0f}", va='center', fontweight='bold')

ax.set_xlabel('Costo Total Canasta Básica ($)', fontsize=12, fontweight='bold')
ax.set_ylabel('Cadena de Supermercados', fontsize=12, fontweight='bold')
ax.set_title(f'Ranking de Supermercados por Costo de Canasta ({fecha_max.strftime("%Y-%m")})', fontsize=14, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x')
ax.invert_yaxis()  # El más barato arriba

plt.tight_layout()
plt.show()

print("\n🏆 Top 5 Supermercados Más Económicos:")
print(df_ranking_reciente.head(5)[['ranking', 'cadena_normalizada', 'costo_total']].to_string(index=False))
print("\n💰 Top 5 Supermercados Más Caros:")
print(df_ranking_reciente.tail(5)[['ranking', 'cadena_normalizada', 'costo_total']].to_string(index=False))

## Resumen Ejecutivo

### Insights Clave del Análisis

In [ ]:
print("="*80)
print("📊 RESUMEN EJECUTIVO - MONITOR DE PRECIOS SIPC")
print("="*80)

print("\n1️⃣ PRECIO PROMEDIO:")
print(f"   - Productos analizados: {df_precio_promedio['producto_id'].nunique():,}")
print(f"   - Precio promedio general: ${df_precio_promedio['precio_promedio'].mean():.2f}")
print(f"   - Rango de precios: ${df_precio_promedio['precio_promedio'].min():.2f} - ${df_precio_promedio['precio_promedio'].max():.2f}")

print("\n2️⃣ VARIACIÓN MENSUAL:")
print(f"   - Variación promedio: {df_variacion['variacion_porcentual'].mean():.2f}%")
print(f"   - Mayor aumento: {df_variacion['variacion_porcentual'].max():.2f}%")
print(f"   - Mayor caída: {df_variacion['variacion_porcentual'].min():.2f}%")

print("\n3️⃣ DISPERSIÓN DE PRECIOS:")
print(f"   - Índice promedio: {df_dispersion['indice_dispersion'].mean():.3f}")
print(f"   - Índice mediano: {df_dispersion['indice_dispersion'].median():.3f}")
print(f"   - Productos con alta dispersión (>1.0): {(df_dispersion['indice_dispersion'] > 1.0).sum():,}")

print("\n4️⃣ CANASTA BÁSICA:")
print(f"   - Cadenas analizadas: {df_canasta['cadena_normalizada'].nunique()}")
print(f"   - Costo promedio: ${df_canasta['costo_canasta'].mean():.2f}")
print(f"   - Rango de costos: ${df_canasta['costo_canasta'].min():.2f} - ${df_canasta['costo_canasta'].max():.2f}")
print(f"   - Diferencia entre más caro y más barato: ${df_canasta['costo_canasta'].max() - df_canasta['costo_canasta'].min():.2f}")

print("\n5️⃣ RANKING SUPERMERCADOS:")
top_economico = df_ranking_reciente.iloc[0]
top_caro = df_ranking_reciente.iloc[-1]
print(f"   - Más económico: {top_economico['cadena_normalizada']} (${top_economico['costo_total']:.0f})")
print(f"   - Más caro: {top_caro['cadena_normalizada']} (${top_caro['costo_total']:.0f})")
print(f"   - Diferencia de precio: ${top_caro['costo_total'] - top_economico['costo_total']:.0f} ({((top_caro['costo_total']/top_economico['costo_total'] - 1) * 100):.1f}%)")

print("\n" + "="*80)
print("✅ Análisis completado exitosamente")
print("="*80)